In [4]:
import pandas as pd
from sklearn.model_selection import train_test_split
import os

# --- Carregue o resultado do seu script de classificação ---
classified_file = '../reports/classificacao_missoes_keywords_v4.json'
try:
    df = pd.read_json(classified_file)
except FileNotFoundError:
    print(f"ERRO: Arquivo de classificação '{classified_file}' não encontrado.")
    print("Por favor, execute o script 'src/classify_by_keywords.py' primeiro.")
    exit()


# --- Preparação do Estrato ---
# Vamos criar uma coluna de estrato para simplificar.
# Usaremos a primeira missão na lista de 'missoes_atribuidas'.
# Se estiver vazio, marcaremos como 'Nao_Classificado'.
df['strata'] = df['missoes_atribuidas'].apply(lambda x: x.split(', ')[0] if x else 'Nao_Classificado')

# Verifique a distribuição para garantir que temos estratos viáveis
print("Distribuição das empresas por estrato inicial:")
print(df['strata'].value_counts())
print("-" * 30)


# --- Cálculo do Tamanho da Amostra ---
population_size = len(df)
# Usando o número recomendado para 95% de confiança e 5% de margem de erro
sample_size = 302 

if population_size < sample_size:
    sample_size = population_size # Garante que não tentemos amostrar mais do que a população
    print(f"AVISO: O tamanho da população ({population_size}) é menor que o tamanho de amostra desejado. Usando todas as empresas.")


# --- Amostragem Estratificada ---
# Precisamos de um DataFrame com apenas os índices e o estrato
df_for_sampling = df[['strata']].copy()

# A função train_test_split é excelente para amostragem estratificada.
# Vamos usá-la para "dividir" o dataset, mas apenas pegaremos a amostra 'test_set'.
# O 'train_set' será o restante das empresas não selecionadas.
try:
    _, sample_df = train_test_split(
        df_for_sampling,
        test_size=sample_size,
        stratify=df_for_sampling['strata'],
        random_state=42 # Para reprodutibilidade
    )
    # Obtenha o DataFrame completo apenas com as amostras selecionadas
    final_sample_to_label = df.loc[sample_df.index]
except ValueError as e:
    print(f"AVISO: Não foi possível realizar a amostragem estratificada. Provavelmente alguns estratos têm poucos membros. {e}")
    print("Realizando amostragem aleatória simples como alternativa.")
    final_sample_to_label = df.sample(n=sample_size, random_state=42)


# --- Verificação Final e Salvamento ---
print(f"Tamanho da amostra final: {len(final_sample_to_label)} empresas.")
print("\nDistribuição na amostra (deve ser proporcional à original se estratificada):")
print(final_sample_to_label['strata'].value_counts())

# Garante que o diretório de destino exista
output_dir = os.path.join('../data', 'processed')
os.makedirs(output_dir, exist_ok=True)

# Salve a lista de empresas a serem rotuladas em um arquivo CSV
output_path = os.path.join(output_dir, 'amostra_para_rotulagem.csv')
final_sample_to_label.to_csv(output_path, index=False, encoding='utf-8-sig')

print(f"\nArquivo com as amostras a serem rotuladas foi salvo em: '{output_path}'")



Distribuição das empresas por estrato inicial:
strata
M4 Transformacao Digital    464
M1 Agro                     262
Nao_Classificado            215
M2 Saude                    203
M3 Infra Mobilidade         203
M5 Bioeconomia Energia       22
M6 Defesa Soberania          16
Name: count, dtype: int64
------------------------------
Tamanho da amostra final: 302 empresas.

Distribuição na amostra (deve ser proporcional à original se estratificada):
strata
M4 Transformacao Digital    101
M1 Agro                      57
Nao_Classificado             47
M2 Saude                     44
M3 Infra Mobilidade          44
M5 Bioeconomia Energia        5
M6 Defesa Soberania           4
Name: count, dtype: int64

Arquivo com as amostras a serem rotuladas foi salvo em: '../data/processed/amostra_para_rotulagem.csv'


preparar pro doccano

In [5]:
import pandas as pd
import json
import os

# --- CONFIGURAÇÃO ---
INPUT_FILE = os.path.join('../data', 'processed', 'amostra_para_rotulagem.csv')
OUTPUT_FILE = os.path.join('../data', 'processed', 'amostra_para_doccano.jsonl')

# Coluna principal que será o texto a ser rotulado
TEXT_COLUMN = 'descricao_organizacao'

# Colunas que você quer visualizar como contexto adicional (metadados)
METADATA_COLUMNS = [
    'nome_organizacao',
    'segmento_atuacao',
    'tecnologias_disruptivas',
]

def main():
    """
    Converte o arquivo CSV de amostra para o formato JSONL que o Doccano
    utiliza para importar texto com metadados.
    """
    print(f"Carregando amostra do arquivo: {INPUT_FILE}")
    
    if not os.path.exists(INPUT_FILE):
        print(f"ERRO: Arquivo de entrada não encontrado em '{INPUT_FILE}'.")
        print("Por favor, execute o script de seleção de amostra primeiro.")
        return

    df = pd.read_csv(INPUT_FILE)
    
    print("Iniciando conversão para o formato JSONL...")
    
    with open(OUTPUT_FILE, 'w', encoding='utf-8') as f:
        for index, row in df.iterrows():
            # Cria um dicionário para os metadados
            metadata = {col: row[col] for col in METADATA_COLUMNS if col in row and pd.notna(row[col])}
            
            # Monta o objeto JSON para a linha atual
            # A chave 'text' é padrão do Doccano para o texto principal
            # A chave 'meta' é padrão do Doccano para os metadados
            json_record = {
                'text': row[TEXT_COLUMN],
                'meta': metadata 
            }
            
            # Escreve o registro JSON como uma nova linha no arquivo
            f.write(json.dumps(json_record, ensure_ascii=False) + '\n')

    print("-" * 50)
    print("Conversão concluída com sucesso!")
    print(f"Arquivo de saída pronto para importação no Doccano: {OUTPUT_FILE}")
    print("No Doccano, importe este arquivo usando o formato 'JSONL'.")

if __name__ == "__main__":
    main()



Carregando amostra do arquivo: ../data/processed/amostra_para_rotulagem.csv
Iniciando conversão para o formato JSONL...
--------------------------------------------------
Conversão concluída com sucesso!
Arquivo de saída pronto para importação no Doccano: ../data/processed/amostra_para_doccano.jsonl
No Doccano, importe este arquivo usando o formato 'JSONL'.


In [4]:
import pandas as pd
import os

# --- CONFIGURAÇÃO ---
# Arquivo com a classificação preliminar de TODAS as empresas
CLASSIFIED_FULL_DATASET_PATH = os.path.join('../reports', 'classificacao_missoes_keywords_v4.json')
# Arquivo com as empresas que você JÁ ROTULOU
LABELED_DATA_PATH = os.path.join('../data', 'processed', 'rotulados.csv')
# Arquivo de saída para a nova leva de rotulagem
OUTPUT_PATH = os.path.join('../data', 'processed', 'amostra_para_rotulagem_batch_2.csv')

# Missões que queremos fortalecer (nosso alvo). M6 foi removida para tratamento manual.
TARGET_MISSIONS = ['M1_Agro', 'M2_Saude', 'M5_Bioeconomia_Energia']
# Número de novas amostras que queremos coletar
N_NEW_SAMPLES = 75

def get_new_samples():
    """
    Identifica e seleciona novas amostras para rotulagem, focando em missões
    específicas e excluindo empresas já rotuladas.
    """
    print("Iniciando a busca por novas amostras para rotulagem (excluindo M6)...")

    # 1. Carregar os datasets necessários
    try:
        df_full = pd.read_json(CLASSIFIED_FULL_DATASET_PATH)
        df_labeled = pd.read_csv(LABELED_DATA_PATH)
    except FileNotFoundError as e:
        print(f"ERRO: Não foi possível encontrar um dos arquivos necessários: {e}")
        print("Verifique se os caminhos CLASSIFIED_FULL_DATASET_PATH e LABELED_DATA_PATH estão corretos.")
        return

    print(f"{len(df_full)} empresas no total, {len(df_labeled)} já rotuladas.")

    # 2. Remover empresas que já foram rotuladas
    labeled_names = df_labeled['Organização'].unique()
    df_unlabeled = df_full[~df_full['nome_organizacao'].isin(labeled_names)].copy()
    print(f"Restam {len(df_unlabeled)} empresas não rotuladas para amostragem.")

    # 3. Filtrar para encontrar os melhores candidatos
    # Criamos uma máscara booleana: um candidato é válido se a sua classificação
    # preliminar (por keywords) for em QUALQUER uma das nossas missões alvo.
    candidate_mask = df_unlabeled[TARGET_MISSIONS].sum(axis=1) > 0
    df_candidates = df_unlabeled[candidate_mask]
    
    if len(df_candidates) == 0:
        print("Nenhum novo candidato encontrado para as missões alvo. Saindo.")
        return
        
    print(f"Encontrados {len(df_candidates)} candidatos para as missões {', '.join(TARGET_MISSIONS)}.")

    # 4. Amostrar a partir dos candidatos
    # Como não estamos mais priorizando a M6, fazemos uma amostragem aleatória simples
    # do nosso pool de candidatos qualificados.
    if len(df_candidates) < N_NEW_SAMPLES:
        print(f"AVISO: O número de candidatos ({len(df_candidates)}) é menor que o tamanho da amostra desejado ({N_NEW_SAMPLES}). Usando todos os candidatos.")
        sample_size = len(df_candidates)
    else:
        sample_size = N_NEW_SAMPLES
        
    final_sample_to_label = df_candidates.sample(n=sample_size, random_state=42)
    
    print("-" * 50)
    print("Amostragem concluída.")
    print(f"Tamanho da nova amostra para rotulagem: {len(final_sample_to_label)} empresas.")
    print("\nDistribuição preliminar da nova amostra (baseada no modelo de keywords):")
    # Mostra a contagem para as missões que estávamos buscando
    print(final_sample_to_label[TARGET_MISSIONS].sum())

    # 5. Salvar o resultado
    # Selecionamos apenas as colunas originais para manter o formato limpo
    output_cols = ['nome_organizacao', 'descricao_organizacao', 'categoria_organizacao', 'segmento_atuacao', 'tecnologias_disruptivas', 'fase_negocio', 'papel_no_ecossistema']
    
    # Garante que todas as colunas existem no dataframe antes de salvar
    final_output_cols = [col for col in output_cols if col in final_sample_to_label.columns]
    
    os.makedirs(os.path.dirname(OUTPUT_PATH), exist_ok=True)
    final_sample_to_label[final_output_cols].to_csv(OUTPUT_PATH, index=False, encoding='utf-8-sig')

    print(f"\nArquivo com as novas amostras a serem rotuladas foi salvo em: '{OUTPUT_PATH}'")


if __name__ == "__main__":
    get_new_samples()


Iniciando a busca por novas amostras para rotulagem (excluindo M6)...
1385 empresas no total, 300 já rotuladas.
Restam 1087 empresas não rotuladas para amostragem.
Encontrados 419 candidatos para as missões M1_Agro, M2_Saude, M5_Bioeconomia_Energia.
--------------------------------------------------
Amostragem concluída.
Tamanho da nova amostra para rotulagem: 75 empresas.

Distribuição preliminar da nova amostra (baseada no modelo de keywords):
M1_Agro                   34
M2_Saude                  36
M5_Bioeconomia_Energia    18
dtype: int64

Arquivo com as novas amostras a serem rotuladas foi salvo em: '../data/processed/amostra_para_rotulagem_batch_2.csv'


In [6]:
import pandas as pd
import os

# --- CONFIGURAÇÃO DE ARQUIVOS ---
# Dataset original com todos os dados rotulados
ORIGINAL_GOLDEN_DATASET_PATH = os.path.join('../data', 'processed', 'golden_dataset.csv')
# Novo batch de dados rotulados que você acabou de criar
NEW_LABELS_BATCH_PATH = os.path.join('../data', 'processed', 'batch2_rotulado.csv')
# Dataset completo com todas as 1385 empresas, para buscar as features de texto
FULL_EXECUTORES_PATH = os.path.join('../data', 'processed', 'df_executores.csv')
# Arquivo de saída com o Golden Dataset combinado e atualizado
COMBINED_GOLDEN_DATASET_PATH = os.path.join('../data', 'processed', 'golden_dataset_v2.csv')

# Mapeamento das colunas do novo batch para o formato padrão
MISSION_MAP = {
    'M1': 'M1_Agro', 'M2': 'M2_Saude', 'M3': 'M3_Infra_Mobilidade',
    'M4': 'M4_Transformacao_Digital', 'M5': 'M5_Bioeconomia_Energia', 'M6': 'M6_Defesa_Soberania'
}

def process_new_batch(df_new):
    """
    Processa o novo batch de dados, convertendo 'Alinha'/'Não Alinha' para 1/0
    e renomeando as colunas.
    """
    # Renomeia a coluna de nome para padronização
    df_new = df_new.rename(columns={'Nome da Organização': 'nome_organizacao'})

    # Itera sobre as colunas de missão (M1, M2, etc.)
    for col in MISSION_MAP.keys():
        if col in df_new.columns:
            # Converte 'Alinha' para 1 e qualquer outro valor ('Não Alinha') para 0
            df_new[col] = df_new[col].apply(lambda x: 1 if str(x).strip() == 'Alinha' else 0)
    
    # Renomeia as colunas de missão para o formato final (e.g., 'M1' -> 'M1_Agro')
    df_new = df_new.rename(columns=MISSION_MAP)
    
    return df_new

def main():
    """
    Função principal para integrar os novos rótulos ao Golden Dataset.
    """
    print("Iniciando a integração dos novos rótulos...")

    # 1. Carregar os datasets
    try:
        df_golden_v1 = pd.read_csv(ORIGINAL_GOLDEN_DATASET_PATH)
        df_batch2 = pd.read_csv(NEW_LABELS_BATCH_PATH)
        df_full = pd.read_csv(FULL_EXECUTORES_PATH)
    except FileNotFoundError as e:
        print(f"ERRO: Não foi possível encontrar um dos arquivos necessários: {e}")
        return

    print(f"Golden Dataset original com {len(df_golden_v1)} registros.")
    print(f"Novo batch com {len(df_batch2)} novos rótulos.")

    # 2. Processar o novo batch de dados
    df_batch2_processed = process_new_batch(df_batch2)
    
    # 3. Buscar as features de texto (descrição, etc.) para o novo batch
    # Usamos um merge com o dataset completo para trazer as colunas que faltam.
    df_batch2_enriched = pd.merge(
        df_batch2_processed,
        df_full,
        on='nome_organizacao',
        how='left'
    )
    
    # Garantir que a ordem das colunas seja a mesma do dataset original
    df_batch2_enriched = df_batch2_enriched[df_golden_v1.columns]

    # 4. Combinar o dataset original com o novo batch enriquecido
    df_golden_v2 = pd.concat([df_golden_v1, df_batch2_enriched], ignore_index=True)
    
    # Remove possíveis duplicatas que possam ter sido adicionadas por engano
    df_golden_v2 = df_golden_v2.drop_duplicates(subset=['nome_organizacao'])
    
    print(f"O novo Golden Dataset combinado tem {len(df_golden_v2)} registros.")

    # 5. Salvar o novo Golden Dataset
    os.makedirs(os.path.dirname(COMBINED_GOLDEN_DATASET_PATH), exist_ok=True)
    df_golden_v2.to_csv(COMBINED_GOLDEN_DATASET_PATH, index=False, encoding='utf-8-sig')

    print("-" * 50)
    print("Integração concluída com sucesso!")
    print(f"Novo Golden Dataset salvo em: '{COMBINED_GOLDEN_DATASET_PATH}'")
    print("\nPróximo passo recomendado: Execute novamente o script 'train_evaluate_model_v4.py',")
    print("ajustando o caminho para carregar este novo arquivo.")

if __name__ == "__main__":
    main()


Iniciando a integração dos novos rótulos...
Golden Dataset original com 310 registros.
Novo batch com 75 novos rótulos.
O novo Golden Dataset combinado tem 374 registros.
--------------------------------------------------
Integração concluída com sucesso!
Novo Golden Dataset salvo em: '../data/processed/golden_dataset_v2.csv'

Próximo passo recomendado: Execute novamente o script 'train_evaluate_model_v4.py',
ajustando o caminho para carregar este novo arquivo.


In [1]:
import pandas as pd
import os

# --- CONFIGURAÇÃO ---
# Caminho para o arquivo com a classificação final de todas as empresas
FINAL_CLASSIFICATION_PATH = os.path.join('../reports', 'classificacao_final_bert.csv')

# Caminho para o novo arquivo que conterá apenas as empresas não classificadas
UNCLASSIFIED_OUTPUT_PATH = os.path.join('../data', 'processed', 'amostra_nao_classificada.csv')

# Nomes das colunas de missão que foram previstas pelo modelo
TARGET_LABELS = ['M1_Agro', 'M2_Saude', 'M3_Infra_Mobilidade', 'M4_Transformacao_Digital', 'M5_Bioeconomia_Energia', 'M6_Defesa_Soberania']

def extract_unclassified_rows():
    """
    Carrega o dataset classificado, encontra as empresas sem nenhuma missão
    atribuída e as salva em um novo arquivo para revisão manual.
    """
    print("Iniciando a extração de empresas não classificadas...")

    # 1. Carregar o dataset completo com as previsões
    try:
        df_final = pd.read_csv(FINAL_CLASSIFICATION_PATH)
        print(f"Arquivo de classificação final com {len(df_final)} empresas carregado.")
    except FileNotFoundError:
        print(f"ERRO: Arquivo de classificação final não encontrado em '{FINAL_CLASSIFICATION_PATH}'.")
        print("Por favor, execute o script 'predict_full_dataset.py' primeiro.")
        return

    # 2. Identificar as linhas não classificadas
    # Somamos os valores das colunas de missão para cada linha.
    # Se a soma for 0, significa que todas as colunas são 0 (não classificada).
    df_unclassified = df_final[df_final[TARGET_LABELS].sum(axis=1) == 0].copy()

    if df_unclassified.empty:
        print("Ótima notícia! Nenhuma empresa não classificada foi encontrada. O modelo atribuiu pelo menos uma missão a todas as empresas.")
        return
    
    print(f"Foram encontradas {len(df_unclassified)} empresas que o modelo não conseguiu classificar.")

    # 3. Preparar o arquivo para análise manual
    # Selecionamos as colunas mais importantes para a sua revisão
    output_columns = [
        'nome_organizacao', 
        'descricao_organizacao', 
        'segmento_atuacao', 
        'tecnologias_disruptivas'
    ]
    # Garante que só selecionamos colunas que realmente existem
    final_output_columns = [col for col in output_columns if col in df_unclassified.columns]
    
    df_to_label = df_unclassified[final_output_columns]

    # 4. Salvar o resultado
    os.makedirs(os.path.dirname(UNCLASSIFIED_OUTPUT_PATH), exist_ok=True)
    df_to_label.to_csv(UNCLASSIFIED_OUTPUT_PATH, index=False, encoding='utf-8-sig')
    
    print("-" * 50)
    print("Processo concluído com sucesso!")
    print(f"O arquivo com as empresas a serem revisadas manualmente foi salvo em: '{UNCLASSIFIED_OUTPUT_PATH}'")


if __name__ == "__main__":
    extract_unclassified_rows()


Iniciando a extração de empresas não classificadas...
Arquivo de classificação final com 1385 empresas carregado.
Foram encontradas 57 empresas que o modelo não conseguiu classificar.
--------------------------------------------------
Processo concluído com sucesso!
O arquivo com as empresas a serem revisadas manualmente foi salvo em: '../data/processed/amostra_nao_classificada.csv'


In [2]:
import pandas as pd
import os
import re

# --- CONFIGURAÇÃO ---
# Caminho para o arquivo com a classificação completa feita pelo modelo BERT
FULL_CLASSIFICATION_PATH = os.path.join('../reports', 'classificacao_final_bert.csv')
# Caminho para o seu novo arquivo com os rótulos manuais
MANUAL_LABELS_PATH = os.path.join('../data', 'processed', 'amostra_nao_classificada_rotulada.csv')
# Caminho para o arquivo de saída final e revisado
FINAL_REVISED_OUTPUT_PATH = os.path.join('../reports', 'classificacao_final_revisada.csv')

# Mapeamento dos nomes das colunas do seu CSV para os nomes padrão do nosso dataset
COLUMN_MAP = {
    'Nome da Organização': 'nome_organizacao',
    'M1 - Agro': 'M1_Agro',
    'M2 - Saúde': 'M2_Saude',
    'M3 - Mobilidade': 'M3_Infra_Mobilidade',
    'M4 - Digital': 'M4_Transformacao_Digital',
    'M5 - Bioeconomia': 'M5_Bioeconomia_Energia',
    'M6 - Defesa': 'M6_Defesa_Soberania'
}

def process_manual_labels(df_manual):
    """
    Processa o DataFrame de rótulos manuais para o formato padrão (1/0).
    """
    # Renomeia as colunas usando o mapa definido
    df_manual = df_manual.rename(columns=COLUMN_MAP)

    # Converte 'Alinha' para 1 e qualquer outro valor para 0
    for col in COLUMN_MAP.values():
        if col != 'nome_organizacao' and col in df_manual.columns:
            # A regex procura pelo emoji de check ✅
            df_manual[col] = df_manual[col].apply(lambda x: 1 if isinstance(x, str) and '✅' in x else 0)
    
    return df_manual

def main():
    """
    Função principal para integrar as revisões manuais ao dataset final.
    """
    print("Iniciando a integração das revisões manuais...")

    # 1. Carregar os datasets
    try:
        df_full_auto = pd.read_csv(FULL_CLASSIFICATION_PATH)
        df_manual = pd.read_csv(MANUAL_LABELS_PATH)
    except FileNotFoundError as e:
        print(f"ERRO: Não foi possível encontrar um dos arquivos necessários: {e}")
        return

    print(f"Carregados {len(df_full_auto)} registros da classificação automática.")
    print(f"Carregados {len(df_manual)} registros da revisão manual.")

    # 2. Processar o arquivo de rótulos manuais
    df_manual_processed = process_manual_labels(df_manual)

    # 3. Atualizar o dataset principal com os rótulos manuais
    # Usaremos o 'nome_organizacao' como chave para encontrar e atualizar as linhas.
    # Definir como índice facilita a atualização.
    df_full_auto.set_index('nome_organizacao', inplace=True)
    df_manual_processed.set_index('nome_organizacao', inplace=True)

    # O método update modifica o DataFrame 'df_full_auto' no local
    df_full_auto.update(df_manual_processed)
    
    # Resetar o índice para que 'nome_organizacao' volte a ser uma coluna
    df_full_auto.reset_index(inplace=True)

    print("Registros originais foram atualizados com sucesso com os rótulos manuais.")

    # 4. Recalcular a coluna de resumo 'missoes_previstas' para refletir as mudanças
    # (vamos renomeá-la para 'missoes_finais' para maior clareza)
    target_labels = list(COLUMN_MAP.values())[1:] # Pega todos os nomes de missão padrão
    
    df_full_auto['missoes_finais'] = df_full_auto[target_labels].apply(
        lambda row: ', '.join([mission.replace('_', ' ') for mission, assigned in row.items() if assigned]),
        axis=1
    )
    # Remove a coluna de previsões antigas se ela existir
    if 'missoes_previstas' in df_full_auto.columns:
        df_full_auto = df_full_auto.drop(columns=['missoes_previstas'])

    # 5. Salvar o resultado final e revisado
    os.makedirs(os.path.dirname(FINAL_REVISED_OUTPUT_PATH), exist_ok=True)
    df_full_auto.to_csv(FINAL_REVISED_OUTPUT_PATH, index=False, encoding='utf-8-sig')

    print("-" * 50)
    print("Processo de integração concluído!")
    print(f"O arquivo final e revisado foi salvo em: '{FINAL_REVISED_OUTPUT_PATH}'")

if __name__ == "__main__":
    main()


Iniciando a integração das revisões manuais...
Carregados 1385 registros da classificação automática.
Carregados 57 registros da revisão manual.
Registros originais foram atualizados com sucesso com os rótulos manuais.
--------------------------------------------------
Processo de integração concluído!
O arquivo final e revisado foi salvo em: '../reports/classificacao_final_revisada.csv'
